In [ ]:
'https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/33538'
'https://www.kaggle.com/frankherfert/local-validation-with-detailed-product-comparison'
    

load the respective training data  
load the respective test data  
add function(s) specific to that dataset  
train each model on the training data  
pickle the trained model (for safe keeping, not totally necessary)  
make predictions on the test data  
pickle the results  

In [1]:
import os

import pandas as pd
import numpy as np

from predict_submit_funcs import submit_fit_score_pred_log

In [2]:
os.chdir('../Data/')

In [3]:
orders = pd.read_csv('orders.csv')
test_ids = orders[orders.eval_set == 'test'].loc[:,['order_id','user_id']]

# Without adding department

In [4]:
df = pd.read_pickle('full_features')

## Logistic Regression

In [50]:

def submit_fit_score_pred_log(df, test_ids):
    """    
    Takes a DataFrame, training, and validation data as its input.
    Returns f1-score, features and their coefficients, and predicted non-re-orders and re-orders.
    """
    
    y = df['in_cart']
    X_pred = df.loc[test_ids.user_id]
    X = df.drop(['product_id','user_id',
                          'latest_cart','in_cart'],axis=1) 
    X_pred.drop(['product_id','user_id',
                    'latest_cart','in_cart'],axis=1,inplace=True)
    
    X_pred.reset_index(drop=True,inplace=True)
    test_ids.reset_index(drop=True,inplace=True)
    
    lr = GaussianNB()
    lr.fit(X, y)
    predictions = pd.DataFrame(lr.predict(X_pred))
    
    output = pd.merge(test_ids,predictions,
                      left_index=True,right_index=True)
    
    return output

In [56]:
y = df['in_cart']
X_pred = df.loc[test_ids.user_id]
X = df.drop(['product_id','user_id',
                      'latest_cart','in_cart'],axis=1) 
X_pred.drop(['product_id','user_id',
                'latest_cart','in_cart'],axis=1,inplace=True)

X_pred.reset_index(drop=True,inplace=True)
test_ids.reset_index(drop=True,inplace=True)

lr = GaussianNB()
lr.fit(X, y)
predictions = pd.DataFrame(lr.predict(X_pred))

output = pd.merge(test_ids,predictions,
                  left_index=True,right_index=True)

In [73]:
df[df.user_id.isin(test_ids.user_id.values.tolist())]

,product_id,user_id,num_ord_per_user_per_prod,latest_cart,in_cart,product_total_orders,product_avg_add_to_cart_order,user_total_orders,user_avg_cartsize,user_total_products,user_avg_days_since_prior_order,user_product_avg_add_to_cart_order,user_product_order_freq,number_orders_since_product_last_ordered,percent_of_time_product_ordered,mean_order_hour_of_day_by_user,diff_between_average_and_current_order_time,max_order_number,recent_order_weight


In [80]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [70]:
test_ids

,order_id,user_id
0,2774568,3
1,329954,4
2,1528013,6
3,1376945,11
4,1356845,12
5,2161313,15
6,1416320,16
7,1735923,19
8,1980631,20
9,139655,22


In [52]:
predictions = submit_fit_score_pred_log(df, test_ids)

In [55]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 3 columns):
order_id    75000 non-null int64
user_id     75000 non-null int64
0           75000 non-null int64
dtypes: int64(3)
memory usage: 1.7 MB


## Gaussian Naive Bayes

## Random Forest

# With Polynomial Features

In [ ]:
df = pd.read_pickle('poly_feats_without_dep_df.pkl')

## Gaussian Naive Bayes

# Adding Departments

In [ ]:
df = pd.read_pickle('full_features_with_dep')

## Gaussian Naive Bayes

## Multinomial Naive Bayes

# After Scaling Data

In [ ]:
df = pd.read_pickle('scaled_feats_df')

## Logistic Regression

## Gaussian Naive Bayes

# Scaled Polynomial Features

In [ ]:
df = pd.read_pickle('scaled_poly_feats_without_dep_df.pkl')

## Logistic Regression

## Gaussian Naive Bayes